# Pokémon TCG Synergy Engine
---

In [1]:
# import libraries for data wrangling
import ast
import re
import pandas as pd
import numpy as np
import src

## Pokémon TCG Data Wrangling
---

In [2]:
data = src.get_card_data()

In [3]:
df_pokemon_cards = src.prep_card_data(data)

In [4]:
df_pokemon_cards

,id,set_name,set_number,supertype,name,stage,is_ex,is_tera,primary_type,evolves_from,...,attack_damage_modifier,attack_cost,cards_needed_for_attack,attack_energy_cost,is_coin_flip,damage_per_energy,retreat_cost,prize_card_value,setup_time,is_immune_to_bench_damage
0,sv1-1,sv1,1,Pokémon,Pineco,Basic,0,0,Grass,NaN,...,1,"[Colorless, Colorless]",2,2,False,5.00,2.0,1,0,0
1,sv1-2,sv1,2,Pokémon,Heracross,Basic,0,0,Grass,NaN,...,1,"[Grass, Colorless]",2,2,False,5.00,2.0,1,0,0
2,sv1-2,sv1,2,Pokémon,Heracross,Basic,0,0,Grass,NaN,...,9,"[Grass, Grass, Colorless]",3,3,False,30.00,2.0,1,0,0
3,sv1-3,sv1,3,Pokémon,Shroomish,Basic,0,0,Grass,NaN,...,1,[Grass],1,1,False,10.00,1.0,1,0,0
4,sv1-4,sv1,4,Pokémon,Breloom,Stage 1,0,0,Grass,Shroomish,...,6,[Grass],2,1,False,60.00,1.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2881,svp-157,svp,157,Pokémon,Zapdos,Basic,0,0,Lightning,NaN,...,2,"[Lightning, Colorless]",2,2,False,10.00,1.0,1,0,0
2882,svp-157,svp,157,Pokémon,Zapdos,Basic,0,0,Lightning,NaN,...,8,"[Lightning, Colorless, Colorless]",3,3,False,26.67,1.0,1,0,0
2883,svp-158,svp,158,Pokémon,Pachirisu,Basic,0,0,Lightning,NaN,...,NaN,[Colorless],1,1,True,NaN,1.0,1,0,0
2884,svp-158,svp,158,Pokémon,Pachirisu,Basic,0,0,Lightning,NaN,...,3,"[Lightning, Colorless]",2,2,False,15.00,1.0,1,0,0


In [5]:
df_pokemon_cards.to_csv('pokemon_cards.csv', index=False)